# FRENCH
## TextBlob

In [ ]:
!pip install -U textblob-fr


In [ ]:

from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle


fr_df_factor = pd.read_pickle('data/fr_df_factor.pkl')
fr_df_factor=fr_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
class_dict=defaultdict(int)
label_dict=defaultdict(int)
subjectivity_dict=defaultdict(int)
sub_label_dict=defaultdict(int)
sentiment=[]
subjectivity=[]
sent_stats=[]
subj_stats=[]
for i in tqdm(range(len(fr_df_factor)), position=0, leave=True):
    
    classes = tb(fr_df_factor.text[i])
    sent=classes.polarity
    sent_stats.append(sent)
    # earlier, we classified it more detailed. For better comparison, we have no slight or strong levels anymore, 
    # just negative, positive and neutral
    '''if sent>=0.5:
        class_dict["strong_positive"]+=1
        label_dict[str("strong_positive_"+str(fr_df_factor.label[i]))]+=1
        sentiment.append("strong_positive")
    elif sent<=-0.5:
        class_dict["strong_negative"]+=1
        label_dict[str("strong_negative_"+str(fr_df_factor.label[i]))]+=1
        sentiment.append("strong_negative")'''
    if sent<=-0.6:
        class_dict["negative"]+=1
        label_dict[str("negative_"+str(fr_df_factor.label[i]))]+=1
        sentiment.append("negative")

    elif sent>=0.6:
        class_dict["positive"]+=1
        label_dict[str("positive_"+str(fr_df_factor.label[i]))]+=1        
        sentiment.append("positive")

        '''elif sent>-0.3 and sent<=-0.1:
        class_dict["slightly_negative"]+=1
        label_dict[str("slightly_negative_"+str(fr_df_factor.label[i]))]+=1
        sentiment.append("slightly_negative")

    elif sent<0.3 and sent>=0.1:
        class_dict["slightly_positive"]+=1
        label_dict[str("slightly_positive_"+str(fr_df_factor.label[i]))]+=1
        sentiment.append("slightly_positive")'''

    else:
        class_dict["neutral"]+=1 
        label_dict[str("neutral_"+str(fr_df_factor.label[i]))]+=1
        sentiment.append("neutral")

    sub=classes.subjectivity
    subj_stats.append(sub)

    if sub>=0.9:

        subjectivity_dict["subjective"]+=1
        sub_label_dict[str("subjective_"+str(fr_df_factor.label[i]))]+=1
        subjectivity.append("subjective")
 
    else:
        subjectivity_dict["not_subjective"]+=1 
        sub_label_dict[str("not_subjective_"+str(fr_df_factor.label[i]))]+=1
        subjectivity.append("not_subjective")
    
fr_df_factor["sentiment"]=sentiment
fr_df_factor["subjectivity"]=subjectivity
fr_df_factor["sentiment_score"]=sent_stats
fr_df_factor["subjectivity_score"]=subj_stats
fr_df_factor.to_pickle('data/fr_df_sentiments.pkl')
print(class_dict.items())
print(label_dict.items())
print(subjectivity_dict.items())
print(sub_label_dict.items())

In [ ]:
len(sent_stats)
print(min(sent_stats))
print(max(sent_stats))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=sent_stats)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=sent_stats,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([-0.35,0.35])

In [ ]:
len(subj_stats)
print(min(subj_stats))
print(max(subj_stats))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=subj_stats)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=subj_stats,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([0,0.05])

In [ ]:
import os
from xml.etree import cElementTree
def load(path):
        """ Loads the XML-file (with sentiment annotations) from the given path.
            By default, Sentiment.path is lazily loaded.
        """
        # <word form="great" wordnet_id="a-01123879" pos="JJ" polarity="1.0" subjectivity="1.0" intensity="1.0" />
        # <word form="damnmit" polarity="-0.75" subjectivity="1.0" label="profanity" />
        
        if not os.path.exists(path):
            return
        words, synsets, labels = {}, {}, {}
        xml = cElementTree.parse(path)
        xml = xml.getroot()
        lex={}
        for w in xml.findall("word"):
            
                w, s  = (
                    w.attrib.get("form"),
                    w.attrib.get("subjectivity", 0.0),
                    
                )
                if float(s)>0.0:
                    lex[w]=s
        
        return lex     
lookup=load("data/fr-sentiment.xml")
print(len(list(lookup.keys())))

In [ ]:
#!pip install wordcloud
textblob_df=pd.read_pickle("data/fr_df_sentiments.pkl")
subj_df=textblob_df[(textblob_df["subjectivity_score"]>=1.0)]
subj_df.reset_index(drop=True, inplace=True)
print(len(subj_df))
from nltk import word_tokenize, sent_tokenize
subjsent=[]
for i in range(len(subj_df)):
    sentences=sent_tokenize(subj_df.loc[i].text, language="french")
    
    for s in sentences:
        token=word_tokenize(s, language="french")
        
        for tok in token:
            if tok.lower() in lookup.keys():
                  subjsent.append(s)

subs=" ".join(list(subjsent))
text = subs
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import os

d = os.path.dirname(__file__) if "__file__" in locals() else os.getcwd()

from nltk.corpus import stopwords



nichtinteressant = ["1", "er", "1er","fr", "B", "ci","X", "juin", "décembre", "août", "février", "mars", "janvier", "avril", "mai", "juillet", "octobre", "novembre", "septembre", "après"]
liste_der_unerwuenschten_woerter = stopwords.words('french')
liste_der_unerwuenschten_woerter= liste_der_unerwuenschten_woerter+nichtinteressant
STOPWORDS.update(liste_der_unerwuenschten_woerter)
wordcloud = WordCloud(background_color="white",width=1920, height=1080).generate(text)

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
print(subjsent[-600:-450])
print(len(subjsent))

In [ ]:
text="compte tenu de son âge - 58 ans au moment des faits - susceptible de l'exposer à de grandes difficultés de réinsertion économique, de la courte durée des relations de travail et de la légèreté de la décision prise."

words=text.split(" ")
for w in words:
    if w in lookup.keys():
        print(w)

In [ ]:
from collections import defaultdict
textblob_df=pd.read_pickle('data/fr_df_sentiments.pkl')

textblob_dict=defaultdict(int)
t_list=[]
for t in textblob_df["subjectivity"]:
    textblob_dict[t]+=1
textblob_dict

In [ ]:
from collections import defaultdict
textblob_df=pd.read_pickle('data/fr_df_sentiments.pkl')

textblob_dict=defaultdict(int)
t_list=[]
for t in textblob_df["sentiment"]:
    textblob_dict[t]+=1
textblob_dict

In [ ]:
from IPython.display import Markdown


fr_df_sentiment=fr_df_factor[(fr_df_factor["sentiment"] == "positive")|(fr_df_factor["sentiment"] == "strong_positive")|(fr_df_factor["sentiment"] == "negative")|(fr_df_factor["sentiment"] == "strong_negative")]



visual=[]

sentences=[]
for i in tqdm(range(len(fr_df_sentiment)), position=0, leave=True):
    
    tokens=fr_df_sentiment.iloc[i]["text"].split()
    if visual:
        sentences.append(visual)
    
    visual=[]
    
    subj=fr_df_sentiment.iloc[i]["subjectivity"]
    if subj=="strongly_subjective":
        visual.append(str("<em><strong>Subjectivity: "+subj+"</em></strong>"))
    elif subj=="subjective":
        visual.append(str("<strong>Subjectivity: "+subj+"</strong>"))
    elif subj=="slightly_subjective":
        visual.append(str("<em>Subjectivity: "+subj+"</em>"))
    
    
 
    
    
    s=fr_df_sentiment.iloc[i]["sentiment"]
    if s=="strong_positive":
        visual.append(str("<code style="+'"background:darkgreen;color:white"'+"> "+s+" </code>"))
    elif s=="strong_negative":
        visual.append(str("<code style="+'"background:darkred;color:white"'+"> "+s+" </code>"))
    elif s=="negative":
        visual.append(str("<code style="+'"background:crimson;color:white"'+"> "+s+" </code>"))
    elif s=="positive":
        visual.append(str("<code style="+'"background:mediumseagreen;color:white"'+"> "+s+" </code>"))
    elif s=="slightly_negative":
        visual.append(str("<code style="+'"background:salmon;color:white"'+"> "+s+" </code>"))
    elif s=="slighly_positive":
        visual.append(str("<code style="+'"background:yellowgreen;color:white"'+"> "+s+" </code>"))
    
    visual.append(str("<strong>Decision: "+str(fr_df_sentiment.iloc[i]["label"]) +"</strong> "))
    
    for t in tokens:
        classes = tb(t)
        sub=classes.subjectivity
        sent=classes.polarity
        if sub>=0.5:
            tok=str("<em><strong>"+t+"</em></strong>")

        elif sub<0.5 and sub>=0.3:
            tok=str("<strong>"+t+"</strong>")
        elif sub<0.3 and sub>=0.1:
            tok=str("<em>"+t+"</em>")
        else:
            tok=t
        
        if sent>=0.5:
            visual.append(str("<code style="+'"background:darkgreen;color:white"'+"> "+tok+" </code>"))
        elif sent<=-0.5:
            visual.append(str("<code style="+'"background:darkred;color:white"'+"> "+tok+" </code>"))
        elif sent>-0.5 and sent<=-0.3:
            visual.append(str("<code style="+'"background:crimson;color:white"'+"> "+tok+" </code>"))
        elif sent<0.5 and sent>=0.3:
            visual.append(str("<code style="+'"background:mediumseagreen;color:white"'+"> "+tok+" </code>"))
        elif sent>-0.3 and sent<=-0.1:
            visual.append(str("<code style="+'"background:salmon;color:white"'+"> "+tok+" </code>"))
        elif sent<0.3 and sent>=0.1:
            visual.append(str("<code style="+'"background:yellowgreen;color:white"'+"> "+tok+" </code>"))

        else:
            visual.append(" "+tok+" ")

import markdown


html_export=[]
    
for s in sentences:
    display(Markdown(" ".join(s)))
    html = markdown.markdown(" ".join(s))
    html_export.append(html)

with open("fr_sentiments.html","w", encoding="UTF-8") as f:
    for ht in html_export:
        f.write(ht)

## DistilCamemBERT

In [ ]:
!pip install sacremoses


In [ ]:

from transformers import pipeline
import textwrap


nlp = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment",max_length=512, truncation=True
)



sp=[]
sn=[]
neuneu=[]

import textwrap
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle

width=512
sentiment_neu=[]
fr_df_factor = pd.read_pickle('data/fr_df_factor.pkl')
fr_df_factor=fr_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)


for i in tqdm(range(len(fr_df_factor)), position=0, leave=True):
    senti_neu=defaultdict(int)
    doc= nlp(textwrap.wrap(fr_df_factor.text[i], width, break_long_words=False))
    
  
    senti_neu["positive"]=0
    senti_neu["negative"]=0
    senti_neu["neutral"]=0
 
    
    for sent in doc:
        lab=sent["label"]
        if lab=="5 stars":
            senti_neu["positive"]+=1
        elif lab == "4 stars":
            senti_neu["neutral"]+=1
        elif lab == "1 star":
            senti_neu["negative"]+=1

        elif lab == "2 stars":
            senti_neu["neutral"]+=1

        else:
            senti_neu["neutral"]+=1
     
    sentiment_neu.append(max(senti_neu, key=senti_neu.get))
   
    sn.append(senti_neu["negative"]/len(list(doc)))
    sp.append(senti_neu["positive"]/len(list(doc)))
    neuneu.append(senti_neu["neutral"]/len(list(doc)))
    
fr_df_factor["sentiment_neu"]=sentiment_neu

fr_df_factor["positive_neu"]=sp
fr_df_factor["negative_neu"]=sn
fr_df_factor["neutral_neu"]=neuneu

  


fr_df_factor.to_pickle('data/fr_df_sentiments_distilcamembert_small_no_dup_3classes.pkl')
fr_df_factor.head()






    
    




## FlauBERT

In [ ]:

import textwrap


from collections import defaultdict
import pandas as pd
import pickle


sp=[]
sn=[]
neuneu=[]


from statistics import mean
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_small_cased")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/flaubert_small_cased_sentiment")

nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, max_length=512, truncation=True)

from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle
width=512

sentiment_neu=[]
fr_df_factor = pd.read_pickle('data/fr_df_factor.pkl')
fr_df_factor=fr_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)


for i in tqdm(range(len(fr_df_factor)), position=0, leave=True):

    senti_neu=defaultdict(int)
    doc= nlp(textwrap.wrap(fr_df_factor.text[i], width, break_long_words=False))
    
  
    
    senti_neu["positive"]=0
    senti_neu["negative"]=0
    senti_neu["neutral"]=0
    
    for sent in doc:
        lab=sent["label"]
        if lab=="very_positive":           
            senti_neu["positive"]+=1
        elif lab == "positive":
            senti_neu["neutral"]+=1
        elif lab == "very_negative":
            senti_neu["negative"]+=1
        elif lab == "negative":
            senti_neu["neutral"]+=1
        else:
            senti_neu["neutral"]+=1
     
    sentiment_neu.append(max(senti_neu, key=senti_neu.get))
   
    sn.append(senti_neu["negative"]/len(list(doc)))
    sp.append(senti_neu["positive"]/len(list(doc)))
    neuneu.append(senti_neu["neutral"]/len(list(doc)))
    

fr_df_factor["sentiment_neu"]=sentiment_neu

fr_df_factor["positive_neu"]=sp
fr_df_factor["negative_neu"]=sn
fr_df_factor["neutral_neu"]=neuneu

  


fr_df_factor.to_pickle('data/fr_df_sentiments_flaubert_small_no_dup_3classes.pkl')
fr_df_factor.head()




    



## VADER

In [ ]:
!pip install vaderSentiment-fr


In [ ]:
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer
import spacy
SIA = SentimentIntensityAnalyzer()
fr_nlp = spacy.load('fr_core_news_lg')
config = {"punct_chars": None}
fr_nlp.add_pipe("sentencizer", config=config)
sentiment=[]

n=[]
p=[]
neu=[]


from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle
vader_scores=[]
sentiment=[]
fr_df_factor = pd.read_pickle('data/fr_df_factor.pkl')
fr_df_factor=fr_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)

for i in tqdm(range(len(fr_df_factor)), position=0, leave=True):
    senti=defaultdict(int)
    senti["positive"]=0
    senti["negative"]=0
    senti["neutral"]=0
    doc=fr_nlp(fr_df_factor.text[i])
    for sentence in doc.sents: 
        vs=SIA.polarity_scores(sentence.text)
    
        sent=vs["compound"]
        vader_scores.append(sent)
        if sent>=0.5:
            senti["positive"]+=1
        elif sent<=-0.5:
            senti["negative"]+=1
        else:
            senti["neutral"]+=1

  
    final_class=max(senti, key=senti.get)
    sentiment.append(final_class)            
        
    p.append(senti["positive"]/len(list(doc.sents)))
    n.append(senti["negative"]/len(list(doc.sents)))
    neu.append(senti["neutral"]/len(list(doc.sents)))
fr_df_factor["vader_sentiment"]=sentiment
fr_df_factor["positive"]=p
fr_df_factor["negative"]=n
fr_df_factor["neutral"]=neu
  

fr_df_factor.to_pickle('data/fr_df_sentiments_vader_no_dup_3classes05.pkl')
fr_df_factor.head()


In [ ]:
len(vader_scores)
print(min(vader_scores))
print(max(vader_scores))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=vader_scores)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=vader_scores,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([-0.3,0.3])

## Inter-Annotator Agreement

In [ ]:
import pandas as pd
import pickle

vader_df=pd.read_pickle('data/fr_df_sentiments_vader_no_dup_3classes05.pkl')

print("vader:")
print(vader_df.groupby('vader_sentiment').count()['id'])





In [ ]:
flaubert_df=pd.read_pickle('data/fr_df_sentiments_flaubert_small_no_dup_3classes.pkl')
print("flaubert: ")
print(print(flaubert_df.groupby('sentiment_neu').count()['id']))

In [ ]:
cam_df=pd.read_pickle("data/fr_df_sentiments_distilcamembert_small_no_dup_3classes.pkl")
print("camembert: ")
print(cam_df.groupby('sentiment_neu').count()['id'])


In [ ]:

textblob_df=pd.read_pickle('data/fr_df_sentiments.pkl')

from sklearn.metrics import cohen_kappa_score
print("Pairwise Cohen Kappa:")
print(str("Flaubert and VADER: "+str(cohen_kappa_score(flaubert_df["sentiment_neu"],vader_df["vader_sentiment"]))))
print(str("Flaubert and DCamemBERT: "+str(cohen_kappa_score(flaubert_df["sentiment_neu"],cam_df["sentiment_neu"]))))
print(str("Flaubert and TextBlob: "+str(cohen_kappa_score(flaubert_df["sentiment_neu"],textblob_df["sentiment"]))))
print(str("VADER and DCamemBERT: "+str(cohen_kappa_score(vader_df["vader_sentiment"],cam_df["sentiment_neu"]))))
print(str("VADER and TextBlob: "+str(cohen_kappa_score(vader_df["vader_sentiment"],textblob_df["sentiment"]))))
print(str("DCamemBERT and TextBlob: "+str(cohen_kappa_score(cam_df["sentiment_neu"],textblob_df["sentiment"]))))



In [ ]:
#fleiss kappa
from nltk import agreement

s1=flaubert_df["sentiment_neu"].tolist()
s2=cam_df["sentiment_neu"].tolist()
s3=vader_df["vader_sentiment"].tolist()
s4=textblob_df["sentiment"].tolist()

formatted_codes = [[1,i,s1[i]] for i in range(len(s1))] + [[2,i,s2[i]] for i in range(len(s2))]  + [[3,i,s3[i]] for i in range(len(s3))]+ [[4,i,s4[i]] for i in range(len(s4))]
ratingtask = agreement.AnnotationTask(data=formatted_codes)
print('Fleiss\'s Kappa:',ratingtask.multi_kappa())